## 1. Setup and Dependencies

First, let's import all required libraries and check our environment.

In [1]:
# Install required packages (uncomment if needed)
# !pip install roboflow ultralytics pandas numpy matplotlib seaborn

import os
import sys
from pathlib import Path

# Add src to path for custom utilities
sys.path.append(str(Path.cwd().parent.parent / 'src'))

print("Environment setup complete!")
print(f"Current working directory: {os.getcwd()}")

Environment setup complete!
Current working directory: /home/aurorax/Git_repos/FishBehaveAI/notebooks/training


## 2. Download Dataset from Roboflow

Configure your Roboflow credentials and download the dataset.

**⚠️ Security Note:** Replace the API key with your own. Consider using environment variables for sensitive data.

In [2]:
from roboflow import Roboflow

# Roboflow configuration
API_KEY = "hPG6x1mG5XJR2YgHc6Eo"  # TODO: Replace with your API key or use os.getenv('ROBOFLOW_API_KEY')
WORKSPACE = "fishbehaviour"
PROJECT_NAME = "prova01_train_videouno"
VERSION = 1
FORMAT = "yolov9"

# Initialize Roboflow
rf = Roboflow(api_key=API_KEY)
print(f"Connected to Roboflow workspace: {WORKSPACE}")

ModuleNotFoundError: No module named 'roboflow'

In [ ]:
# Download the dataset
project = rf.workspace(WORKSPACE).project(PROJECT_NAME)
version = project.version(VERSION)
dataset = version.download(FORMAT)

print(f"\n✅ Dataset downloaded successfully!")
print(f"Dataset location: {dataset.location}")

## 3. Explore Dataset Structure

Let's examine what we downloaded and validate the dataset structure.

In [ ]:
import yaml
from pathlib import Path

# Find and load data.yaml
data_yaml_path = Path(dataset.location) / "data.yaml"

with open(data_yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

print("Dataset Configuration:")
print("=" * 50)
for key, value in data_config.items():
    print(f"{key}: {value}")

In [ ]:
# Count images in each split
def count_files_in_dir(directory, extension='.jpg'):
    """Count files with specific extension in directory"""
    path = Path(directory)
    if path.exists():
        return len(list(path.glob(f'*{extension}'))) + len(list(path.glob(f'*.png')))
    return 0

dataset_path = Path(dataset.location)

splits = {
    'train': dataset_path / 'train' / 'images',
    'valid': dataset_path / 'valid' / 'images',
    'test': dataset_path / 'test' / 'images'
}

print("\nDataset Statistics:")
print("=" * 50)
for split_name, split_path in splits.items():
    if split_path.exists():
        img_count = count_files_in_dir(split_path)
        label_path = split_path.parent / 'labels'
        label_count = count_files_in_dir(label_path, '.txt')
        print(f"{split_name.upper():6s}: {img_count} images, {label_count} labels")
    else:
        print(f"{split_name.upper():6s}: Directory not found")

## 4. Visualize Sample Images

Let's visualize some images with their annotations to understand our data better.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import random

def visualize_sample_with_labels(image_path, label_path, class_names):
    """Visualize an image with its YOLO format bounding boxes"""
    # Load image
    img = Image.open(image_path)
    img_width, img_height = img.size
    
    # Create figure
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(img)
    
    # Read labels
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            labels = f.readlines()
        
        # Draw bounding boxes
        for label in labels:
            parts = label.strip().split()
            if len(parts) >= 5:
                class_id = int(parts[0])
                x_center, y_center, width, height = map(float, parts[1:5])
                
                # Convert YOLO format to pixel coordinates
                x = (x_center - width/2) * img_width
                y = (y_center - height/2) * img_height
                w = width * img_width
                h = height * img_height
                
                # Draw rectangle
                rect = patches.Rectangle((x, y), w, h, linewidth=2, 
                                        edgecolor='lime', facecolor='none')
                ax.add_patch(rect)
                
                # Add label
                class_name = class_names.get(class_id, f"Class {class_id}")
                ax.text(x, y-5, class_name, color='lime', fontsize=10,
                       bbox=dict(boxstyle='round', facecolor='black', alpha=0.5))
    
    ax.axis('off')
    ax.set_title(f"Sample: {image_path.name}")
    plt.tight_layout()
    return fig

# Get class names from data config
class_names_list = data_config.get('names', [])
class_names = {i: name for i, name in enumerate(class_names_list)}

print(f"Classes in dataset: {class_names}")

In [ ]:
# Visualize 3 random samples from training set
train_images_dir = dataset_path / 'train' / 'images'
train_labels_dir = dataset_path / 'train' / 'labels'

if train_images_dir.exists():
    image_files = list(train_images_dir.glob('*.jpg')) + list(train_images_dir.glob('*.png'))
    samples = random.sample(image_files, min(3, len(image_files)))
    
    for img_path in samples:
        label_path = train_labels_dir / f"{img_path.stem}.txt"
        fig = visualize_sample_with_labels(img_path, label_path, class_names)
        plt.show()
else:
    print("Training images directory not found!")

## 5. Configure Training Parameters

Set up the training configuration for YOLO model.

In [ ]:
from ultralytics import YOLO

# Training configuration
TRAINING_CONFIG = {
    'model': 'yolov9c.pt',  # Pretrained model
    'data': str(data_yaml_path),
    'epochs': 50,
    'imgsz': 640,
    'batch': 16,
    'patience': 100,
    'device': None,  # None = auto-detect GPU/CPU
    'project': 'results/training_runs',
    'name': 'yolo_fish_behavior',
    'exist_ok': False,
    'pretrained': True,
    'optimizer': 'auto',
    'verbose': True,
    'seed': 0,
    'deterministic': True,
    'lr0': 0.01,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'cos_lr': False,
    'val': True,
    'plots': True,
    'save': True
}

print("Training Configuration:")
print("=" * 50)
for key, value in TRAINING_CONFIG.items():
    print(f"{key:20s}: {value}")

## 6. Load Model and Start Training

Initialize the YOLO model and begin training.

In [ ]:
# Load pretrained model
model = YOLO(TRAINING_CONFIG['model'])

print(f"✅ Model loaded: {TRAINING_CONFIG['model']}")
print(f"Starting training with {TRAINING_CONFIG['epochs']} epochs...")
print(f"This may take a while depending on your hardware.\n")

In [ ]:
# Train the model
results = model.train(
    data=TRAINING_CONFIG['data'],
    epochs=TRAINING_CONFIG['epochs'],
    imgsz=TRAINING_CONFIG['imgsz'],
    batch=TRAINING_CONFIG['batch'],
    patience=TRAINING_CONFIG['patience'],
    device=TRAINING_CONFIG['device'],
    project=TRAINING_CONFIG['project'],
    name=TRAINING_CONFIG['name'],
    exist_ok=TRAINING_CONFIG['exist_ok'],
    pretrained=TRAINING_CONFIG['pretrained'],
    optimizer=TRAINING_CONFIG['optimizer'],
    verbose=TRAINING_CONFIG['verbose'],
    seed=TRAINING_CONFIG['seed'],
    deterministic=TRAINING_CONFIG['deterministic'],
    lr0=TRAINING_CONFIG['lr0'],
    lrf=TRAINING_CONFIG['lrf'],
    momentum=TRAINING_CONFIG['momentum'],
    weight_decay=TRAINING_CONFIG['weight_decay'],
    warmup_epochs=TRAINING_CONFIG['warmup_epochs'],
    val=TRAINING_CONFIG['val'],
    plots=TRAINING_CONFIG['plots'],
    save=TRAINING_CONFIG['save']
)

print("\n✅ Training completed!")

## 7. Evaluate Model Performance

Run validation on the test set and analyze results.

In [ ]:
# Validate on test set
metrics = model.val(
    data=TRAINING_CONFIG['data'],
    split='test',  # Use test split
    conf=0.5,
    save_json=True,
    plots=True
)

print("\nTest Set Metrics:")
print("=" * 50)
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

## 8. Visualize Training Results

Display training curves and performance metrics.

In [ ]:
import pandas as pd
from pathlib import Path

# Load training results
results_dir = Path(TRAINING_CONFIG['project']) / TRAINING_CONFIG['name']
results_csv = results_dir / 'results.csv'

if results_csv.exists():
    df_results = pd.read_csv(results_csv)
    df_results.columns = df_results.columns.str.strip()  # Clean column names
    
    # Plot training metrics
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss curves
    if 'train/box_loss' in df_results.columns:
        axes[0, 0].plot(df_results['epoch'], df_results['train/box_loss'], label='Box Loss')
        axes[0, 0].plot(df_results['epoch'], df_results['train/cls_loss'], label='Class Loss')
        axes[0, 0].plot(df_results['epoch'], df_results['train/dfl_loss'], label='DFL Loss')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].set_title('Training Losses')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
    
    # mAP curves
    if 'metrics/mAP50(B)' in df_results.columns:
        axes[0, 1].plot(df_results['epoch'], df_results['metrics/mAP50(B)'], label='mAP50')
        axes[0, 1].plot(df_results['epoch'], df_results['metrics/mAP50-95(B)'], label='mAP50-95')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('mAP')
        axes[0, 1].set_title('Validation mAP')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
    
    # Precision and Recall
    if 'metrics/precision(B)' in df_results.columns:
        axes[1, 0].plot(df_results['epoch'], df_results['metrics/precision(B)'], label='Precision')
        axes[1, 0].plot(df_results['epoch'], df_results['metrics/recall(B)'], label='Recall')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Score')
        axes[1, 0].set_title('Precision and Recall')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
    
    # Learning rate
    if 'lr/pg0' in df_results.columns:
        axes[1, 1].plot(df_results['epoch'], df_results['lr/pg0'])
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Learning Rate')
        axes[1, 1].set_title('Learning Rate Schedule')
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Full results saved in: {results_dir}")
else:
    print(f"Results file not found at: {results_csv}")

## 9. Test Inference on Sample Images

Run predictions on test images to visualize model performance.

In [ ]:
# Load best model weights
best_model_path = results_dir / 'weights' / 'best.pt'

if best_model_path.exists():
    model_best = YOLO(str(best_model_path))
    print(f"✅ Loaded best model from: {best_model_path}")
    
    # Run inference on test images
    test_images_dir = dataset_path / 'test' / 'images'
    if test_images_dir.exists():
        test_images = list(test_images_dir.glob('*.jpg')) + list(test_images_dir.glob('*.png'))
        samples = random.sample(test_images, min(3, len(test_images)))
        
        for img_path in samples:
            # Predict
            results = model_best.predict(source=str(img_path), conf=0.5, save=False)
            
            # Display result
            result_plot = results[0].plot()
            plt.figure(figsize=(12, 8))
            plt.imshow(result_plot)
            plt.axis('off')
            plt.title(f"Inference: {img_path.name}")
            plt.tight_layout()
            plt.show()
    else:
        print("Test images directory not found!")
else:
    print(f"Best model weights not found at: {best_model_path}")

## 10. Summary and Next Steps

**Training Complete!** 

Model weights saved in: `{results_dir}/weights/`
- `best.pt`: Best model based on validation metrics
- `last.pt`: Final epoch model

**Next Steps:**
1. Analyze confusion matrix in the results directory
2. Export model for deployment if needed
3. Run behavioral analysis on tracked fish
4. Create Markov chain analysis from predictions

**Files Generated:**
- Training results and plots in `results/training_runs/`
- Model weights in `weights/` subdirectory
- Validation predictions and metrics